# Testing Virtual Zarr with RiOMar and Icechunk

```{warning}
This notebook is a test and demonstrate that it currently does not work to use VirtualZarr with Icechunk with data on Pangeo-EOSC
```


### Notebook

- Justus Magin (author), CNRS-LOPS (France), @keewis

## Bibliography and other interesting resources
- [Icechunk](https://icechunk.io)
- [Zarr](http://zarr.readthedocs.io)
- [RioMar](https://coast.ifremer.fr/Laboratoires-Environnement-Ressources/LER-Pertuis-Charentais-La-Tremblade/Projets/RIOMAR-2024-2030)

In [ ]:
from functools import partial

import fsspec
import virtualizarr
import xarray as xr

fs = fsspec.filesystem("http")

In [ ]:
inroot = "https://data-fair2adapt.ifremer.fr/riomar/GAMAR"
urls = fs.glob(f"{inroot}/*.nc")

In [ ]:
import distributed

cluster = distributed.LocalCluster(n_workers=2)
client = cluster.get_client()
client

In [ ]:
func = partial(
    virtualizarr.open_virtual_dataset,
    backend=virtualizarr.readers.hdf.HDFVirtualBackend,
    indexes={},
    loadable_variables=[
        "time_counter",
        "time_instant",
        "x_rho",
        "y_rho",
        "x_u",
        "x_v",
        "y_u",
        "y_v",
        "axis_nbounds",
    ],
    decode_times=True,
)

futures = client.map(func, urls[:2])
dss = client.gather(futures)
dss

In [ ]:
ds = xr.concat(
    dss,
    dim="time_counter",
    compat="override",
    coords="minimal",
    combine_attrs="drop_conflicts",
).set_coords(["time_counter_bounds", "time_instant_bounds"])
ds

In [ ]:
import icechunk

In [ ]:
storage = icechunk.local_filesystem_storage("riomar.zarr")
storage

In [ ]:
repo = icechunk.Repository.open_or_create(storage)
repo

In [ ]:
session = repo.writable_session("main")
store = session.store
store

In [ ]:
ds.virtualize.to_icechunk(store)

In [ ]:
session.commit("first two datasets")

In [ ]:
rs = repo.readonly_session(branch="main")
rs

In [ ]:
xr.open_dataset(rs.store, engine="zarr", chunks={}, consolidated=False)